<a href="https://colab.research.google.com/github/kryogenica/MERA_Wavelet_Tensor_Network/blob/master/1D_Tensor_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construction of Binary Unitary Circuits
**This notebook intends to convert all the math found in the following paper: *[Representation and design of wavelets using unitary circuits](https://arxiv.org/pdf/1605.07312.pdf)*  into python functions/code. [This notebook is currently under construction.] **


Constructed by Bryant E. Avila

In [0]:
#Numpy will help dealing with arrays/tensors.
#Math will aid trig functions and values.
import numpy as np
import math as Mh

**We start by creating the variables from equations 16 and 17 and setting thier values.**


\begin{align}
t_{1} = 2 + \sqrt{3},  t_{2} = \frac{1}{\sqrt{3}} \tag{16}\\
\theta_{1} = \frac{5\pi}{12},  \theta_{2} = \frac{\pi}{\sqrt{6}} \tag{17}
\end{align}

In [0]:
#Here we set the the variables that our unitary matrix will be using.
t_1 = ( 2 + Mh.sqrt(3) )
t_2 = ( 1 / Mh.sqrt(3) )


theta_1 = (5*Mh.pi)/12
theta_2 = Mh.pi/6

**Next, the unitary gates of the binary unitray circuit is constructed.
Below is a function that will return a binary 2x2 matrix as refered to equation 3. Cos function excluded.**


\begin{align}
u(\theta_{k}) &\equiv \begin{bmatrix}
    cos(\theta_{k})       & sin(\theta_{k})  \\
    -sin(\theta_{k})      & cos(\theta_{k})  
\end{bmatrix},\notag\\
&\quad= \cos(\theta_{k}) \begin{bmatrix}
    1           &t_{k}  \\
    -t_{k}  &1     &   
\end{bmatrix}.\notag\\ \tag{3}
\end{align}

In [0]:
#Function returns the unitary matrix Uk with the numerical value tk established 
def unitary_matrix(t_k):
  u_k = np.array([[1,t_k],[-1*t_k,1]])
  return u_k

**Here we call on the above function and get in return the 2x2 unitary gate with its tk value established.**

In [0]:
u1 = unitary_matrix(t_1)
u2 = unitary_matrix(t_2)
one = np.identity(1)

**It is useful to construct a function that will allow us to easily direct sum any amount of rank 2 tensors of arbitrary dimension.**

In [0]:
#Direct Sum function for N 2D arrays. [[:]] AND [[:],[:],...]
def direct_sum (*argv):
  initial = argv[0]
  for arg in argv[1:]:
    shape_1 = np.asanyarray(initial.shape)
    shape_2 = np.asanyarray(arg.shape)
  
    zeros_1 = np.zeros((shape_2[0],shape_1[1]))
    zeros_2 = np.zeros((shape_1[0],shape_2[1]))

    sub_d_s_1 = np.vstack((initial,zeros_1))
    sub_d_s_2 = np.vstack((zeros_2,arg))
  
    sub_direct_sum = np.hstack((sub_d_s_1,sub_d_s_2))
    initial = sub_direct_sum
  return initial

**Using the python function above the unitary circuit depicted below (purple highlight) will be manually constructed.**


![MultiscaleCircuit](https://drive.google.com/uc?export=view&id=1T7eGN4Wi9CXknVbsPo5L70VmAVpcdG9K)


\begin{align}
Fig. 2
\end{align}

In [0]:
#The bottom layer of Fig. 2b is constructed.
u2_DS_u2 = direct_sum(u2,u2)
print("u2 ⊕ u2:")
print (u2_DS_u2)
print ()

#The top layer of Fig. 2b is constructed.
one_DS_u1_DS_one = direct_sum(one,u1,one)
print("1 ⊕ u1 ⊕ 1:")
print (one_DS_u1_DS_one)


u2 ⊕ u2:
[[ 1.          0.57735027  0.          0.        ]
 [-0.57735027  1.          0.          0.        ]
 [ 0.          0.          1.          0.57735027]
 [ 0.          0.         -0.57735027  1.        ]]

1 ⊕ u1 ⊕ 1:
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          3.73205081  0.        ]
 [ 0.         -3.73205081  1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [0]:
#Here we stitch the bottom and lower layer. 
unitary_circuit = np.matmul(u2_DS_u2,one_DS_u1_DS_one)
print (unitary_circuit)

[[ 1.          0.57735027  2.15470054  0.        ]
 [-0.57735027  1.          3.73205081  0.        ]
 [ 0.         -3.73205081  1.          0.57735027]
 [ 0.          2.15470054 -0.57735027  1.        ]]


**With the unitary circuit now constructed we can now obtain the scaling sequence h by the equation below or Fig.2C** 

$$
\begin{align}
 \langle {\bf {U}} \times \mathbb{1}_{r}  \rangle = \textbf{h} = (u_{2}
 \oplus u_{2}) \times (1 \oplus u_{1} \oplus 1) \times \lbrack0,0,1,0\rbrack ^\dagger   \tag{10 & 11}\\
\end{align}
$$

$$
\begin{align}
 \langle {\bf {U}} \times \mathbb{1}_{r}  \rangle = \tag{10 & 11}\\
\end{align}
$$

In [0]:
odd_one = [0,0,1,0]
h = np.matmul(unitary_circuit,odd_one)
print (h)

[ 2.15470054  3.73205081  1.         -0.57735027]


**The above result does not seem to match the result of equation 18 (see below). We need to normalize our result for this:**


\begin{align}
\bf {h} \approx \begin{bmatrix}
    0.48296\\
    0.83651\\
    0.22414\\
    -0.12940
\end{bmatrix},    
\bf{g} \approx \begin{bmatrix}
    0.12940\\
    0.22414\\
    -0.83651\\
    0.48296
\end{bmatrix}\tag{18}\\
\end{align}

In [0]:
h_squared = np.matmul(h,h)
h_norm = Mh.sqrt(np.sum(h_squared))
print (h/h_norm)

[ 0.48296291  0.8365163   0.22414387 -0.12940952]


**Now lets recover the wavelet sequence *g* from the unitary circuit by multiplying it with the even one vector.**

In [0]:
even_one = [0,1,0,0]
g = np.matmul(unitary_circuit,even_one)
print (g)

[ 0.57735027  1.         -3.73205081  2.15470054]


**Again lets normalize this sequence to compare it with its result in equation 18.**

In [0]:
g_squared = np.matmul(g,g)
g_norm = Mh.sqrt(np.sum(g_squared))
print (g/g_norm)

[ 0.12940952  0.22414387 -0.8365163   0.48296291]


**It will be usefull to not have to construct the binary uniatary circuit ourselves. The python function below will aid us in this. It needs to have the lenght of the bottom portion of the circuit give and the unitary gates in their respective stacking order. Here the depth of the circuit is restricted by the number of unitary gates.**

In [0]:
def b_unit_construct (M,*othersss):
  size = M*othersss[0].shape[0]
  A = np.zeros((M,size,size))
  i = 0
  for other in othersss:
    Uk = other
    for m in range(int(0+i),int(M-1)):
      Uk = direct_sum(Uk,other)
    if i==0:
      A[i,:,:] = Uk
    else:
      ones = np.identity(i)
      Uk = direct_sum(ones,Uk,ones)
      A[i,:,:] = Uk
    del Uk
    i += 1
  
  Result = A[0,:,:]
  for m in range(1,M):
    Result = np.matmul(Result,A[m,:,:])
  return Result
      

**Lets show this works by generating the scaling sequence again.**

In [0]:
unitary_circuit = b_unit_construct(2,u2,u1)
h = np.matmul(unitary_circuit,odd_one)
print (np.round(h/h_norm,5))#using h_norm calculated from before

[ 0.48296  0.83652  0.22414 -0.12941]


In [0]:
%%html
<marquee style='width: 30%; color: green;'><b>Success!</b></marquee>

# D6 Daubechies scaling sequence
In order to obtain a 6 Daubechies sequence with a binary unitary circuit it is neccesary to have a 3 depth circuit. In general, in order to obtain a 2N Daubechies sequence a N depth circuit is needed. Let's an example below. 


**One can find that the family of t_k values for a 3 depth unitary circuit are:**


\begin{align}
t_{1} = 0.105889,  t_{2} = -1.83118,  t_{3} = -0.412287 \tag{Solution.1}\\
\ t_{1} = 9.44381, t_{2} = 1.83118, t_{3} = 0.412287 \tag{Solution.2}
\end{align}


**Solution.1 is used for the following demonstartion**

In [0]:
#Solution.1
t_1 = 0.105889
t_2 = -1.83118
t_3 = -0.412287

#Solution.2
#t_1 = 9.44381
#t_2 = 1.83118
#t_3 = 0.412287

u1 = unitary_matrix(t_1)
u2 = unitary_matrix(t_2)
u3 = unitary_matrix(t_3)

**Lets create a 3 depth unitary circuit in order to obtain the D6 Daubechies scaling sequence.**

In [0]:
unitary_circuit_3 = b_unit_construct(3,u3,u2,u1)
print (unitary_circuit_3)

[[ 1.         -0.412287    0.75497171  0.0799432   0.          0.        ]
 [ 0.412287    1.         -1.83118    -0.19390182  0.          0.        ]
 [ 0.          1.83118     1.04365666 -0.306398    0.75497171  0.        ]
 [ 0.          0.75497171  0.306398    1.04365666 -1.83118     0.        ]
 [ 0.          0.         -0.19390182  1.83118     1.         -0.412287  ]
 [ 0.          0.         -0.0799432   0.75497171  0.412287    1.        ]]


**The D6 Daubechies sequence is:**

In [0]:
#Using the result from above
even_one_3 = [0,0,0,1,0,0]
h6 = np.matmul(unitary_circuit_3,even_one_3)

h6_squared = np.matmul(h6,h6)
h6_norm = Mh.sqrt(np.sum(h6_squared))
print (h6/h6_norm)

[ 0.03522619 -0.08544094 -0.13501129  0.45987711  0.80689158  0.33267091]


**Let's compre these with a 
[standard result.](https://en.wikipedia.org/wiki/Daubechies_wavelet)
Wikipedia states the D6 sequence is: **

~ [0.4705, 1.1911, 0.6504, -0.1909, -0.1208, 0.0498]

**These are clearly not normalize, lets do so and compare...**


In [0]:
f = np.asanyarray([0.47046721, 1.19111692, 0.650365, -0.19093442, -0.12083221, 0.0498175])
f_squared = np.matmul(f,f)
f_norm = Mh.sqrt(np.sum(f_squared))
print (f/f_norm)

[ 0.32337673  0.81871698  0.44702989 -0.13123922 -0.0830543   0.03424217]


They are the same sequence! (Diferent order and slighty off ). We can say that these unitary circuits can construct wavelet scaling sequence to any power 

-------Under constrcution------

In [0]:
#Generating a random unitary vector
np.random.seed(2)
rand_array = np.random.uniform(0,1,size=(4,))
rand_dot = np.multiply(rand_array,rand_array)
norm_factor = Mh.sqrt(np.sum(rand_dot))
rand_normal = rand_array/norm_factor
print ("Random unitary vector: "+str(rand_normal))
print ("Self dot product of random unitary vector: "+str(np.matmul(rand_normal,rand_normal)))

Random unitary vector: [0.52779162 0.03138488 0.66539138 0.52697752]
Self dot product of random unitary vector: 0.9999999999999998


In [0]:
R = np.matmul(rand_normal,unitary_circuit)
print (R)

[ 0.50967155 -1.01169019  1.61550363  0.91114141]


In [0]:
Rprime = R
Rprime_dot = np.multiply(Rprime,Rprime)
norm_factor = Mh.sqrt(np.sum(Rprime_dot))
Rprime_normal = Rprime/norm_factor
print (Rprime_normal)
print (np.matmul(Rprime_normal,Rprime_normal))

rand_prime = np.matmul(unitary_circuit,Rprime_normal)
print (rand_prime)
rand_prime_dot = np.multiply(rand_prime,rand_prime)
norm_factor = Mh.sqrt(np.sum(rand_prime_dot))
rand_normal_prime = rand_prime/norm_factor
print (rand_normal_prime)

[ 0.23451306 -0.46550482  0.74333501  0.41923973]
0.9999999999999998
[ 1.56741807  2.17326301  2.72267083 -1.01294843]
[ 0.39660667  0.54990472  0.68892239 -0.2563082 ]


# Circuit Construction Algorithim
If given a 2N sequence of scaling coefficients there is a way to encode them into the θ values of a N depth binary uitary circuit.